In [1]:
import numpy as np 
import cv2

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import StandardScaler , PolynomialFeatures 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error


import tensorflow as tf



np.set_printoptions(precision=2)

tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)


In [2]:


x = np.load('feature.npy', allow_pickle=True)
y = np.load('person.npy', allow_pickle=True)

# x = np.expand_dims(x, axis=1)
y = np.expand_dims(y, axis=1)

# Check the shape of x
print(f"Original shape of x: {x.shape}")

# Reshape x to the desired shape
x = np.reshape(x, (x.shape[0], -1))

print(y.shape)
print(x.shape)





Original shape of x: (98, 100, 100)
(98, 1)
(98, 10000)


In [3]:
x_train,x_,y_train,y_ = train_test_split(x,y,test_size=0.4,random_state=1)

x_cv, x_test,y_cv,y_test = train_test_split(x_,y_,test_size=0.5,random_state=1)

del x_, y_




In [4]:

Scaler = StandardScaler()

poly = PolynomialFeatures( 1 , include_bias=False)

x_train_mapped = poly.fit_transform(x_train)
x_cv_mapped = poly.transform(x_cv)
x_test_mapped = poly.transform(x_test)

x_train_mapped_scaled = Scaler.fit_transform(x_train_mapped)
x_cv_mapped_scaled = Scaler.transform(x_cv_mapped)
x_test_mapped_scaled = Scaler.transform(x_test_mapped)



In [5]:
model = Sequential([
    
    Dense(25,activation= 'relu'),
    Dense(15,activation='relu'),
    Dense(10,activation='relu'),
    Dense(1,activation='linear')
    
],
  name= 'model'
  )



In [6]:

# Ensure the numpy arrays have a consistent data type
x_train_mapped_scaled = x_train_mapped_scaled.astype(np.float32)
x_cv_mapped_scaled = x_cv_mapped_scaled.astype(np.float32)
y_train = y_train.astype(np.float32)
y_cv = y_cv.astype(np.float32)

model.compile(
    loss='mse',
    optimizer='adam',
)

model.fit(x_train_mapped_scaled, y_train, epochs=10, verbose=0)

yhat = model.predict(x_cv_mapped_scaled)
cv_mse = mean_squared_error(y_cv, yhat)/2

print(cv_mse)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1.1499146223068237


In [7]:
'''

people = ['klay thompson' ,'kyrie irving', 'lebron james','SGA']
text =''

input_data = np.expand_dims(x_test_mapped_scaled[0], axis=0)
predicted_y = model.predict(input_data)
print(people[int(y_test[0])])

if -0.5<=predicted_y<=0.5 :text =people[0]
elif 0.5<predicted_y<=1.5 :text =people[1]
elif 1.5<predicted_y<=2.5 :text =people[2]
elif 2.5<predicted_y<=3.5 :text =people[3]

img =np.reshape(input_data,(1,100,100))
print(img.shape)

img=input_data.astype(np.uint8)

img =cv2.putText(img ,text,(0,img.shape[1]-10),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,255,0),1)
img

cv2.imshow('predicted image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

"\n\npeople = ['klay thompson' ,'kyrie irving', 'lebron james','SGA']\ntext =''\n\ninput_data = np.expand_dims(x_test_mapped_scaled[0], axis=0)\npredicted_y = model.predict(input_data)\nprint(people[int(y_test[0])])\n\nif -0.5<=predicted_y<=0.5 :text =people[0]\nelif 0.5<predicted_y<=1.5 :text =people[1]\nelif 1.5<predicted_y<=2.5 :text =people[2]\nelif 2.5<predicted_y<=3.5 :text =people[3]\n\nimg =np.reshape(input_data,(1,100,100))\nprint(img.shape)\n\nimg=input_data.astype(np.uint8)\n\nimg =cv2.putText(img ,text,(0,img.shape[1]-10),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,255,0),1)\nimg\n\ncv2.imshow('predicted image',img)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n"

In [11]:
import numpy as np
import cv2

people = ['klay thompson', 'kyrie irving', 'lebron james', 'SGA']
text = ''

# Assuming x_test_mapped_scaled and model are already defined
input_data = np.expand_dims(x_test_mapped_scaled[0], axis=0)
predicted_y = model.predict(input_data)
print(people[int(y_test[0])])  # Corrected this line

if -0.5 <= predicted_y <= 0.5:
    text = people[0]
elif 0.5 < predicted_y <= 1.5:
    text = people[1]
elif 1.5 < predicted_y <= 2.5:
    text = people[2]
elif 2.5 < predicted_y <= 3.5:
    text = people[3]


img = x_test[0].reshape((100, 100))  
img = img.astype(np.uint8)

# Add text to the image
cv2.putText(img, text, (0, img.shape[1] - 10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 255, 0), 1)

# Write the image to a file
cv2.imwrite('image.jpg', img)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
kyrie irving


C:\Users\a2\AppData\Local\Temp\ipykernel_14776\230226907.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(people[int(y_test[0])])  # Corrected this line


True

In [13]:
final_pic = cv2.imread('image.jpg')
cv2.imshow('predicted image', final_pic)
cv2.waitKey(0)
cv2.destroyAllWindows()